In [1]:
import os
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm import tqdm


c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 2. Load dữ liệu văn bản đã được làm sạch

In [ ]:
# 3. Chuẩn bị mô hình BERT

In [ ]:
# 4. Sinh vector đặc trưng (embeddings) cho toàn bộ văn bản

In [ ]:
# 5. Huấn luyện mô hình Machine Learning (Train Logistic Regression và Random Forest)

In [ ]:
# 6. Đánh giá kết quả (Tính Accuracy và F1-score)

In [ ]:
# 7. Lưu kết quả huấn luyện và lưu kết quả thành results_bert.json

🔹 Cross-validation trên tập train:
Logistic Regression       | F1_macro: 0.8695 ± 0.0135 | Accuracy: 0.8669 ± 0.0134
Random Forest (depth=10)  | F1_macro: 0.9067 ± 0.0047 | Accuracy: 0.9066 ± 0.0039
Random Forest (depth=20)  | F1_macro: 0.9064 ± 0.0066 | Accuracy: 0.9066 ± 0.0065

📊 Evaluation on test set:

--- Logistic Regression ---
Accuracy: 0.8647058823529412
F1-weighted: 0.8651153781185208
              precision    recall  f1-score   support

    COVID-19       0.89      0.84      0.86       266
      NORMAL       0.91      0.89      0.90       188
   PNEUMONIA       0.81      0.88      0.84       226

    accuracy                           0.86       680
   macro avg       0.87      0.87      0.87       680
weighted avg       0.87      0.86      0.87       680


--- Random Forest ---
Accuracy: 0.8955882352941177
F1-weighted: 0.8954539399885812
              precision    recall  f1-score   support

    COVID-19       0.90      0.92      0.91       266
      NORMAL       0.90     

FileNotFoundError: [Errno 2] No such file or directory: './models/TF-IDF/results_tfidf.json'